In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta, date

In [2]:
def job_openings():
    month = ['202005']
    job = []
    k = {}
    d = []
    c = []

    for m in month:

        url = 'http://www.jobkorea.co.kr/starter/calendar/{}'.format(m)
        params={
            'Sel_Date':m
        }
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text,'lxml')
        body = soup.find('tbody')
        trs = body.find_all('tr')
        for tr in trs:
            divs = tr.find_all('div',class_='calCont')

            for div in divs:
                span = div.find('span',class_="link start ")
                if span == None:
                    continue
                day = div.find('strong',class_='day').text
                company = span.find('a').text[2:].strip()
                c.append(company)
                if len(day)==1:
                    day = '0'+day
                d.append(m[0:4]+'-'+m[4:]+'-'+str(day))
        df1 = pd.DataFrame({'날짜':d, '회사':c})
        
    return df1

In [3]:
def data_join(df1):
    date_range = (pd.date_range(start='20200501', end='20200530')).strftime("%Y-%m-%d").tolist()
    df2=pd.DataFrame({'날짜':date_range})
    df_OUTER_JOIN = pd.merge(df1, df2, left_on='날짜', right_on='날짜', how='outer', sort=True)
    df_finally = df_OUTER_JOIN.fillna('None')
    return df_finally

In [4]:
def last_day_of_month(ds):
    cur_ds = datetime.strptime(ds, '%Y%m')
    next_month = datetime(year=cur_ds.year, month=cur_ds.month+1, day=1)
    last_day_month = next_month - timedelta(days=1)
    return datetime.strftime(last_day_month, '%Y%m%d')

In [5]:
def append_month_info(month,month_end_day):
    job,d,c = [],[],[]
    k = {}
    url = 'http://www.jobkorea.co.kr/starter/calendar/{}'.format(month)
    params={
        'Sel_Date':month
    }
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text)
    body = soup.find('tbody')
    trs = body.find_all('tr')
    for tr in trs:
        divs = tr.find_all('div',class_='calCont')

        for div in divs:   
            span = div.find('span',class_="link start ")
            if span == None:
                continue
            day = div.find('strong',class_='day').text
            company = span.find('a').text[2:].strip()
            c.append(company)
            if len(day)==1:
                day = '0'+day
            d.append(month[0:4]+'-'+month[4:]+'-'+str(day))
    df_curr_1 = pd.DataFrame({'회사':c,'날짜':d})
    date_range = (pd.date_range(start=month+'01', end=month_end_day)).strftime("%Y-%m-%d").tolist()
    df_curr_2=pd.DataFrame({'날짜':date_range})
    df_OUTER_JOIN = pd.merge(df_curr_1, df_curr_2, left_on='날짜', right_on='날짜', how='outer', sort=True)
    df_curr = df_OUTER_JOIN.fillna('None')
    return df_curr

In [6]:
df1 = job_openings()
df = data_join(df1)
last_day_of_month = last_day_of_month(date.today().strftime("%Y%m%d")[:-2])
df_curr = append_month_info(date.today().strftime("%Y%m%d")[:-2],last_day_of_month)
df = df.append(df_curr,ignore_index=True)
df.to_csv('job_crawling.csv',encoding='utf-8', index=False)
print('=====END Job Crawling=====')

=====END Job Crawling=====


C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
